# Latent Dirichlet Allocation (LDA)

🎯 The goal of this challenge is to find topics within a corpus of emails with the **LDA** algorithm (Unsupervised Learning in NLP)

✉️ Here is a collection of 1K+ ***unlabelled emails***. Let's try to ***extract topics*** from them!

In [5]:
import pandas as pd
import string
import re
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_validate
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [2]:
import pandas as pd

url = 'https://wagon-public-datasets.s3.amazonaws.com/05-Machine-Learning/10-Natural-Language-Processing/lda_data'

data = pd.read_csv(url, sep=",", header=None)
data.columns = ['text']
data.head()

,text
0,From: gld@cunixb.cc.columbia.edu (Gary L Dare)...
1,From: atterlep@vela.acs.oakland.edu (Cardinal ...
2,From: miner@kuhub.cc.ukans.edu\nSubject: Re: A...
3,From: atterlep@vela.acs.oakland.edu (Cardinal ...
4,From: vzhivov@superior.carleton.ca (Vladimir Z...


In [3]:
data.shape

(1199, 1)

## (1) Preprocessing 

❓ **Question (Cleaning**) ❓ You're used to it by now... Clean up! Store the cleaned text in a new column "clean_text" of the DataFrame.

In [6]:
def make_lower_case(sentence):
    return sentence.lower()


def remove_punctuation(sentence):
    for punctuation in string.punctuation:
        sentence = sentence.replace(punctuation, "")
    return sentence


def remove_numbers(sentence):
    pattern = r"[0-9]"
    return re.sub(pattern, "", sentence)


def tokenize(sentence):
    return word_tokenize(sentence)

def remove_whitespance(tokenized_sentence):
    return [word for word in tokenized_sentence if word]


def remove_stopwords(tokenized_sentence):
    sw = set(stopwords.words("english"))
    return [word for word in tokenized_sentence if word not in sw]


def lemmatize(tokenized_sentence):
    lemmatizer = WordNetLemmatizer()
    v_lemmatized = [lemmatizer.lemmatize(word, pos="v") for word in tokenized_sentence]

    v_n_lemmatized = [lemmatizer.lemmatize(word, pos="n") for word in v_lemmatized]

    v_n_a_lemmatized = [lemmatizer.lemmatize(word, pos="a") for word in v_n_lemmatized]

    v_n_a_r_lemmatized = [
        lemmatizer.lemmatize(word, pos="r") for word in v_n_a_lemmatized
    ]

    return v_n_a_r_lemmatized


def preprocessing(sentence, remove_stopwords=True):
    sentence = make_lower_case(sentence)
    sentence = remove_numbers(sentence)
    sentence = remove_punctuation(sentence)
    tokenized_sentence = tokenize(sentence)
    if remove_stopwords:
        tokenized_sentence = remove_stopwords(tokenized_sentence)
    tokenized_sentence = remove_whitespance(tokenized_sentence)
    tokenized_sentence = lemmatize(tokenized_sentence)
    sentence = " ".join(tokenized_sentence)
    return sentence

In [10]:
data["text_clean"] = data["text"].apply(preprocessing, remove_stopwords=False)
data["text_clean"].head(1)

0    from gldcunixbcccolumbiaedu gary l dare subjec...
Name: text_clean, dtype: object

## (2) Latent Dirichlet Allocation model

❓ **Question (Training)** ❓ Train a LDA model to extract potential topics

In [ ]:
# YOUR CODE HERE

##  (3) Visualize potential topics

🎁 We coded for you a  function that prints the words associated with the potential topics.

In [55]:
def print_topics(model, vectorizer):
    for idx, topic in enumerate(model.components_):
        print("Topic %d:" % (idx))
        print([(vectorizer.get_feature_names_out()[i], topic[i])
                        for i in topic.argsort()[:-10 - 1:-1]])

❓ **Question** ❓ Print the topics extracted by your LDA.

In [56]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import set_config

set_config("diagram")

vectorizer = TfidfVectorizer()

vectorized_documents = vectorizer.fit_transform(data["text_clean"])
vectorized_documents = pd.DataFrame(
    vectorized_documents.toarray(),
    columns = vectorizer.get_feature_names_out()
)

vectorized_documents.head(1)


,aa,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaauuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuugggggggggggggggg,aacc,aadams,aafreenetcarletonca,aargh,aaron,aaronbinahccbrandeisedu,aaroncathenamitedu,aassists,...,zombo,zone,zoo,zoom,zorasterism,zubov,zupancic,zurich,zwart,zzzzzz
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
# Instantiate the LDA
n_components = 2
lda_model = LatentDirichletAllocation(n_components=n_components, max_iter = 100)

# Fit the LDA on the vectorized documents
lda_model.fit(vectorized_documents)

LatentDirichletAllocation(max_iter=100, n_components=2)

In [58]:
document_topic_mixture = lda_model.transform(vectorized_documents)
document_topic_mixture

array([[0.04441539, 0.95558461],
       [0.04367116, 0.95632884],
       [0.04282228, 0.95717772],
       ...,
       [0.04906854, 0.95093146],
       [0.08286657, 0.91713343],
       [0.05209101, 0.94790899]])

In [59]:
topic_word_mixture = pd.DataFrame(
    lda_model.components_,
    columns = vectorizer.get_feature_names_out()
)
topic_word_mixture

,aa,aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaauuuuuuuuuuuuuuuuuuuuuuuuuuuuuuuugggggggggggggggg,aacc,aadams,aafreenetcarletonca,aargh,aaron,aaronbinahccbrandeisedu,aaroncathenamitedu,aassists,...,zombo,zone,zoo,zoom,zorasterism,zubov,zupancic,zurich,zwart,zzzzzz
0,0.504461,0.502275,0.500878,0.500116,0.505991,0.505679,0.506386,0.504876,0.506367,0.503840,...,0.504796,0.506511,0.506499,0.505246,0.501462,0.507282,0.503583,0.503486,0.508052,0.504210
1,0.869156,0.588366,0.527609,0.505227,2.027975,1.212647,1.666984,0.924544,0.835125,0.573883,...,0.744201,2.427500,0.653455,0.743066,0.564495,1.706467,0.583756,0.649412,0.823793,0.664064


## (4) Predict the document-topic mixture of a new text

❓ **Question (Prediction)** ❓

Now that your LDA model is fitted, you can use it to predict the topics of a new text.

1. Vectorize the example
2. Use the LDA on the vectorized example to predict the topics

In [60]:
example = ["My team performed poorly last season. Their best player was out injured and only played one game"]

In [61]:
example_vectorized = vectorizer.transform(example)

lda_vectors = lda_model.transform(example_vectorized)

print("topic 0 :", lda_vectors[0][0])
print("topic 1 :", lda_vectors[0][1])

topic 0 : 0.12418322466447397
topic 1 : 0.875816775335526


/home/branchedelac/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but LatentDirichletAllocation was fitted with feature names
  warnings.warn(
